In [2]:
!pip install opencv-python
!pip install numpy
!pip install matplotlib
!pip install pyperclip

In [3]:
# -*- coding: utf-8 -*-
"""
Created on Tue Apr 11 14:34:34 2017

@author: ukalwa
"""
from __future__ import print_function
import cv2
import numpy as np
import pyperclip as pc
import ast


win_name = "VSS App"

# List of sigs
listOfHSV_Sigs = {}
NUM_OF_HSV_SIGS = 8

DEFAULT_SIG = [0, 11, 90, 255, 187, 255, False]


def dummy(x):
    return x


def onmouse(event, x, y, flags, params):
    global point
    if event == cv2.EVENT_LBUTTONDOWN:
        point = (x, y)
        print(x, y)


point = None
prev_point = None


def watershed_transform(image, mask):
    # noise removal
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=2)

    # sure background area
    sure_bg = cv2.dilate(opening, kernel, iterations=3)

    # Finding sure foreground area
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    ret, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)

    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)
    # Marker labelling
    ret, markers = cv2.connectedComponents(sure_fg)

    # Add one to all labels so that sure background is not 0, but 1
    markers = markers + 1

    # Now, mark the region of unknown with zero
    markers[unknown == 255] = 0
    markers = cv2.watershed(image, markers)
    image[markers == -1] = [255, 0, 0]
    return image

'''
def detect_object_single_frame(image, res, point, prev_point):
    res = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
    cnts, hierarchy = cv2.findContours(res, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    area = np.array([cv2.contourArea(k) for k in cnts])
    print(cnts)
    #cnts = sum([sublist for sublist in cnts], [])
    #cnts = np.array(cnts).tolist()
    area = area[np.logical_and(area > 100, area < 300)]

    mask2 = np.zeros(res.shape)
    cv2.drawContours(mask2, cnts, -1, (255, 255, 255), -1)
    image_copy = np.copy(image)
    if point is not None and point != prev_point:
        for cnt in np.arange(len(cnts)):
            dist = cv2.pointPolygonTest(cnts[cnt], point, True)
            if dist > 0:
                print(area[cnt])
                point = prev_point
    for cnt in np.arange(len(cnts)):
        rect = cv2.boundingRect(cnts[cnt])
        x, y, w, h = rect
        centre_x = round(x + (w / 2))
        centre_y = round(y + (h / 2))

        startPoint = (centre_x - w, centre_y - h)
        endPoint = (centre_x + w, centre_y + h)
        #            print w,h
        cv2.rectangle(image_copy, startPoint, endPoint, (255, 0, 0), 2)
    return image_copy, len(cnts)
'''

def detect_object(image, res, threshold, hsvRange, rectColor, objectArray):
    res = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
    res = cv2.blur(res, (5,5))

    cnts, hierarchy = cv2.findContours(res, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in np.arange(len(cnts)):
        rect = cv2.boundingRect(cnts[cnt])
        x, y, w, h = rect
        if w*h >= threshold*100:
            cv2.rectangle(res, (x, y), (x + w, y + h), (255, 255, 255), -1)


    cnts, hierarchy = cv2.findContours(res, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    mask2 = np.zeros(res.shape)
    cv2.drawContours(mask2, cnts, -1, (255, 255, 255), -1)

    for cnt in np.arange(len(cnts)):
        rect = cv2.boundingRect(cnts[cnt])
        x, y, w, h = rect
        objectArray.append([rect, hsvRange])
        #print(objBoxArray)
        if w*h >= threshold*100:
            cv2.rectangle(image, (x, y), (x + w, y + h), rectColor, 2)
    return len(cnts)

def setup(savedSignatures = {}):

    # easy assigments
    cv2.namedWindow(win_name, cv2.WINDOW_NORMAL)
    cv2.setMouseCallback(win_name, onmouse)

    cv2.createTrackbar('H Low', win_name, 11, 255, dummy)
    cv2.createTrackbar('H High', win_name, 11, 255, dummy)
    cv2.createTrackbar('S Low', win_name, 90, 255, dummy)
    cv2.createTrackbar('S High', win_name, 255, 255, dummy)
    cv2.createTrackbar('V Low', win_name, 187, 255, dummy)
    cv2.createTrackbar('V High', win_name, 255, 255, dummy)
    cv2.createTrackbar('Object Sensitivity', win_name, 0, 10, dummy)
    cv2.createTrackbar('Signature', win_name, 0, NUM_OF_HSV_SIGS - 1, dummy)

    for i in range(0, NUM_OF_HSV_SIGS):
        listOfHSV_Sigs[i] = DEFAULT_SIG

    '''if savedSignatures.__sizeof__() > 0:
        for i in range(0, NUM_OF_HSV_SIGS):
            print(savedSignatures.get(i))
            listOfHSV_Sigs[i][0] = savedSignatures[i][0]
            listOfHSV_Sigs[i][1] = savedSignatures[i][1]
            listOfHSV_Sigs[i][2] = savedSignatures[i][2]
            listOfHSV_Sigs[i][3] = savedSignatures[i][3]
            listOfHSV_Sigs[i][4] = savedSignatures[i][4]
            listOfHSV_Sigs[i][5] = savedSignatures[i][5]

    print(listOfHSV_Sigs, "\n\n\n\n")
    
    '''

    for sig in range(0, NUM_OF_HSV_SIGS - 1):
        cv2.setTrackbarPos('H Low', win_name, listOfHSV_Sigs.get(sig)[0])
        cv2.setTrackbarPos('H High', win_name, listOfHSV_Sigs.get(sig)[1])
        cv2.setTrackbarPos('S Low', win_name, listOfHSV_Sigs.get(sig)[2])
        cv2.setTrackbarPos('S High', win_name, listOfHSV_Sigs.get(sig)[3])
        cv2.setTrackbarPos('V Low', win_name, listOfHSV_Sigs.get(sig)[4])
        cv2.setTrackbarPos('V High', win_name, listOfHSV_Sigs.get(sig)[5])


def stepVSS(webcamFeed):

    global lastSigAccessed

    try:
        lastSigAccessed
    except:
        lastSigAccessed = 0

    ret, image = webcamFeed.read()
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Gets the current signature
    currentSig = cv2.getTrackbarPos('Signature', win_name)

    #if not listOfHSV_Sigs.get(currentSig)[6]:
    #    listOfHSV_Sigs.get(currentSig)[6] = True

    # read trackbar positions for all
    hul = cv2.getTrackbarPos('H Low', win_name)
    huh = cv2.getTrackbarPos('H High', win_name)
    sal = cv2.getTrackbarPos('S Low', win_name)
    sah = cv2.getTrackbarPos('S High', win_name)
    val = cv2.getTrackbarPos('V Low', win_name)
    vah = cv2.getTrackbarPos('V High', win_name)
    obj_threshold = cv2.getTrackbarPos('Object Sensitivity', win_name)

    if currentSig != lastSigAccessed:
        # Pulls data from hashmap so the bars go to their saved position
        cv2.setTrackbarPos('H Low', win_name, listOfHSV_Sigs.get(currentSig)[0])
        cv2.setTrackbarPos('H High', win_name, listOfHSV_Sigs.get(currentSig)[1])
        cv2.setTrackbarPos('S Low', win_name, listOfHSV_Sigs.get(currentSig)[2])
        cv2.setTrackbarPos('S High', win_name, listOfHSV_Sigs.get(currentSig)[3])
        cv2.setTrackbarPos('V Low', win_name, listOfHSV_Sigs.get(currentSig)[4])
        cv2.setTrackbarPos('V High', win_name, listOfHSV_Sigs.get(currentSig)[5])

    # Sets the signature that is active to be whatever the current trackbars values are.
    listOfHSV_Sigs[currentSig] = [hul, huh, sal, sah, val, vah]

    lastSigAccessed = currentSig

    objectList = []

    for sig in range(0, NUM_OF_HSV_SIGS):

        #if listOfHSV_Sigs.get(sig)[6]:

            # make arrays for final values

            # H Low, S Low, V Low
            hsv_low = np.array([listOfHSV_Sigs.get(sig)[0], listOfHSV_Sigs.get(sig)[2], listOfHSV_Sigs.get(sig)[4]])
            hsv_high = np.array([listOfHSV_Sigs.get(sig)[1], listOfHSV_Sigs.get(sig)[3], listOfHSV_Sigs.get(sig)[5]])

            # apply the range on a mask
            mask = cv2.inRange(hsv_image, hsv_low, hsv_high)
            res = cv2.bitwise_and(image, image, mask=mask)

            hsv_range = (listOfHSV_Sigs.get(sig)[0], listOfHSV_Sigs.get(sig)[1], listOfHSV_Sigs.get(sig)[2],
                        listOfHSV_Sigs.get(sig)[3], listOfHSV_Sigs.get(sig)[4], listOfHSV_Sigs.get(sig)[5])

            count = detect_object(image, res, obj_threshold, hsv_range,(sig*31, 255-sig*31, sig*31), objectList)

    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(image, 'Press q to quit', (10, 30), font, 1, (255, 0, 0), 2)

    cv2.imshow("test", image)
    key = cv2.waitKey(5) & 0xFF
    if key == ord('q'):
        webcamFeed.release()
        cv2.destroyAllWindows()
    if key == ord('s'):
        pc.copy(str(listOfHSV_Sigs))

    return objectList



In [4]:
import cv2

def getBySize(objArray, comparitive_size = 0):

    if len(objArray) == 0:
        return None
    if len(objArray) == 1:
        #print(objArray[0][0][0])
        return objArray[0][0][0]

    #print(objArray)

    if comparitive_size > len(objArray) - 1:
        comparitive_size = len(objArray) - 1

    # Insertion Sort

    # Starts from the second element
    for i in range(1, len(objArray)):

        rectToMove = objArray[i]

        # i is only used here
        rectToLeftIndex = i - 1

        # While the RTLI var is greater than 0 and won't throw an access error and
        # while the RTLI area > rectToMove area, which checks if the box to the left of
        # the RTM is greater than RTM. If so, it moves it over one. Keeps doing this until
        # RTLI is less than RTM, which means it's sorted.
        while (rectToLeftIndex >= 0 and objArray[rectToLeftIndex][0][2]*objArray[rectToLeftIndex][0][3] >
               rectToMove[0][2]*rectToMove[0][3] ):

            # Shifts rectangle to the left
            objArray[rectToLeftIndex + 1] = objArray[rectToLeftIndex]

            # Shifts index to the left
            rectToLeftIndex -= 1

        # Shifts the rectangle to the right location
        objArray[rectToLeftIndex + 1] = objArray[rectToLeftIndex]

    # Makes a new array the inverse of objArray, so biggest is 0. I don't get the syntax, but it's whatever.
    newObjArray = objArray[::-1]

    #print(newObjArray[0][0][comparitive_size])
    return newObjArray[comparitive_size][0]

def getByHSV_Sig(objArray, HSV_sig_param):

    returnObjArray = []

    hsv_sig = listOfHSV_Sigs[HSV_sig_param]

    if len(objArray) == 0:
        return None

    #print(len(objArray))

    for i in range(0, len(objArray)-1):
        print("HSV For Loop: ", i)
        print("   ", objArray[i][1], "    ", hsv_sig)

        if (objArray[i][1][0] == hsv_sig[0] and
            objArray[i][1][1] == hsv_sig[1] and
            objArray[i][1][2] == hsv_sig[2] and
            objArray[i][1][3] == hsv_sig[3] and
            objArray[i][1][4] == hsv_sig[4] and
            objArray[i][1][5] == hsv_sig[5]):

            print("Found matching sig")
            if len(objArray) == 1:
                return objArray[i][0]
            returnObjArray.append(objArray[i][0])


    if len(returnObjArray) == 0:
        return None

    # Insertion Sort

    print ("Insertion Sort")

    # Starts from the second element
    for i in range(1, len(returnObjArray)):

        print("Iteration: ", i)
        rectToMove = returnObjArray[i]

        # i is only used here
        rectToLeftIndex = i - 1

        # While the RTLI var is greater than 0 and won't throw an access error and
        # while the RTLI area > rectToMove area, which checks if the box to the left of
        # the RTM is greater than RTM. If so, it moves it over one. Keeps doing this until
        # RTLI is less than RTM, which means it's sorted.
        while (rectToLeftIndex >= 0 and returnObjArray[rectToLeftIndex][2] * returnObjArray[rectToLeftIndex][3] >
               rectToMove[2] * rectToMove[3]):
            # Shifts rectangle to the left
            returnObjArray[rectToLeftIndex + 1] = returnObjArray[rectToLeftIndex]

            # Shifts index to the left
            rectToLeftIndex -= 1

        # Shifts the rectangle to the right location
        returnObjArray[rectToLeftIndex + 1] = returnObjArray[rectToLeftIndex]

        print(returnObjArray)

    # Makes a new array the inverse of objArray, so biggest is 0. I don't get the syntax, but it's whatever.
    returnObjArray = returnObjArray[::-1]

    print(returnObjArray)

    return returnObjArray

def getObjCnt(objArray, HSV_sig = None):
    return len(objArray)

def getCameraWidth(camera):
    return camera.get(cv2.CAP_PROP_FRAME_WIDTH)

def getCameraHeight(camera):
    return camera.get(cv2.CAP_PROP_FRAME_HEIGHT)





In [5]:
# -*- coding: utf-8 -*-
"""
Program heavily influenced by Github user Ukalwa who made a blob detection algorithim for his game.

Link to project I used code from:
https://github.com/ukalwa/blob_color_detection/tree/master

"""

import cv2

vid = cv2.VideoCapture(0)
bool_read, image = vid.read()
frame = 0
if bool_read:

    setup()

    while True:

        objArray = stepVSS(vid)

        HSVBox = getByHSV_Sig(objArray, 0)

        #print(HSVBox)

        if HSVBox is not None:
            x, y, width, height = HSVBox[0]
            print(width * height)





cv2.destroyAllWindows()
vid.release()




HSV For Loop:  0
    (0, 11, 90, 255, 187, 255)      [0, 11, 90, 255, 187, 255]
Found matching sig
HSV For Loop:  1
    (0, 11, 90, 255, 187, 255)      [0, 11, 90, 255, 187, 255]
Found matching sig
HSV For Loop:  2
    (0, 11, 90, 255, 187, 255)      [0, 11, 90, 255, 187, 255]
Found matching sig
HSV For Loop:  3
    (0, 11, 90, 255, 187, 255)      [0, 11, 90, 255, 187, 255]
Found matching sig
HSV For Loop:  4
    (0, 11, 90, 255, 187, 255)      [0, 11, 90, 255, 187, 255]
Found matching sig
HSV For Loop:  5
    (0, 11, 90, 255, 187, 255)      [0, 11, 90, 255, 187, 255]
Found matching sig
HSV For Loop:  6
    (0, 11, 90, 255, 187, 255)      [0, 11, 90, 255, 187, 255]
Found matching sig
HSV For Loop:  7
    (0, 11, 90, 255, 187, 255)      [0, 11, 90, 255, 187, 255]
Found matching sig
HSV For Loop:  8
    (0, 11, 90, 255, 187, 255)      [0, 11, 90, 255, 187, 255]
Found matching sig
HSV For Loop:  9
    (0, 11, 90, 255, 187, 255)      [0, 11, 90, 255, 187, 255]
Found matching sig
HSV For Lo

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
